In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Set color map to have light blue background
sns.set()
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor as VIF
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.inspection import permutation_importance
import warnings
from utils import *
%matplotlib inline

In [ ]:
# load data
df = pd.read_csv('BeijingPM20100101_20151231.csv')

print('INFO:')
df.info()

print("====================")
print('HEAD:')
print(df.head())

print("====================")
# if a column is numeric, show the number of nan values in it
for col_name in df.columns:
    if np.issubdtype(df[col_name].dtype, np.number): 
        print("Column {} is numeric, #nan is {}".format(col_name, np.count_nonzero(np.isnan(df[col_name].values))))
    else:
        print("Column {} is not numeric".format(col_name))

print("====================")
print('DESCRIBE:')
df.describe(include='all')

In [ ]:
# drop columns of no use here
df = df.drop(columns=['No', 'season', 'precipitation'])

In [ ]:
# four PM columns
df_pms = df[['PM_Dongsi', 'PM_Dongsihuan', 'PM_Nongzhanguan', 'PM_US Post']]
# drop nan values from the four columns
df_pms = df_pms.dropna()
# plot the correlation matrix of the four pm columns
ax_pms = sns.heatmap(df_pms.corr(), vmin=-1.0, vmax=1.0, annot=True, fmt='.2g', center=0.0, xticklabels=True, yticklabels=True)
# ax_pms.figure.set_size_inches(8,8)

In [ ]:
# pair plot of the four pm columns
# suppress warnings
with warnings.catch_warnings(record=True):
    sns.pairplot(df_pms, diag_kind='kde')

In [ ]:
# Calculate the mean of the four columns as the final PM 2.5 level
df_pm = df[['PM_Dongsi', 'PM_Dongsihuan', 'PM_Nongzhanguan', 'PM_US Post']]
print("====================")
df_pm.info()
# mean of the four PM columns with valid values
df_pm_mean = df_pm.mean(axis=1)
print("====================")
df_pm_mean.info()
df['PM'] = df_pm_mean
# drop the four original PM 2.5 columns
df = df.drop(columns=['PM_Dongsi', 'PM_Dongsihuan', 'PM_Nongzhanguan', 'PM_US Post'])
# add final PM 2.5 column
df["PM"] = df_pm_mean
print("====================")
df.info()

In [ ]:
# N: 0, NE: 45, E: 90, SE: 135, S: 180, SW: 225, W: 270, NW: 315
direction_dict = {
    'N': 0,
    'NE': 45,
    'E': 90,
    'SE': 135,
    'S': 180,
    'SW': 225,
    'W': 270,
    'NW': 315
}
sin_dict = {k: np.sin(direction_dict[k] / 180 * np.pi) for k in direction_dict.keys()}
cos_dict = {k: np.cos(direction_dict[k] / 180 * np.pi) for k in direction_dict.keys()}
print(sin_dict)
print(cos_dict)

df_cbwd = df['cbwd']

# some rows get 'cv' in this column, I find it hard to assign the sine and cosine terms for these rows
# so the 'cv' rows are assigned 0 in the sine column and the cosine column
def sin_func(x):
    if x == 'cv':
        return 0
    if x == 'NA' or x != x:
        return np.nan
    return sin_dict[x]

def cos_func(x):
    if x == 'cv':
        return 0
    if x == 'NA' or x != x:
        return np.nan
    return cos_dict[x]

def is_cv(x):
    if x == 'cv':
        return 1
    else:
        return 0

# add sine and cosine columns
df['sin_cbwd'] = df_cbwd.apply(sin_func)
df['cos_cbwd'] = df_cbwd.apply(cos_func)
# add is_cv column
df['is_cv'] = df_cbwd.apply(is_cv)
# drop the original 'cwbd' column
df = df.drop(columns=['cbwd'])
df.info()

In [ ]:
# now drop rows with nan values
df = df.dropna()
print("INFO:")
df.info()
print("====================")
print("DESCRIBE:")
df.describe()

In [ ]:
# show data histograms
df.hist(bins=30, figsize=(10, 10))
plt.show()

# Drop cases with extremely high PM 2.5 levels
df = df[df['PM'] <= 500]

print("INFO:")
df.info()
print("====================")
print("DESCRIBE:")
df.describe()

In [ ]:
df_mean = df.mean(axis=0)
df_std = df.std(axis=0)
print('mean:')
print(df_mean)
print("====================")
print('std:')
print(df_std)
# standardization
for col_name in df.columns:
    if col_name != 'PM' and col_name != 'is_cv' and col_name != 'sin_cbwd' and col_name != 'cos_cbwd':
        df[col_name] = df[col_name].apply(lambda x: (x - df_mean[col_name]) / df_std[col_name])
df.describe()

In [ ]:
# plot correlation matrix
corr_matrix = df.corr()
print(corr_matrix)
ax = sns.heatmap(df.corr(), vmin=-1.0, vmax=1.0, annot=True, fmt='.2g', center=0.0, xticklabels=True, yticklabels=True)
ax.figure.set_size_inches(16,16)

In [ ]:
# pair plot
# suppress warnings
with warnings.catch_warnings(record=True):
    sns.pairplot(df, diag_kind='kde')

In [ ]:
# simple linear regression with all features
formula_str = 'PM ~ '
num_feats_added = 0
for i in range(len(df.columns)):
    col_name = df.columns[i]
    if col_name != 'PM':
        if num_feats_added != 0:
            formula_str += ' + '
        formula_str += col_name
        num_feats_added += 1

print(formula_str)
model_simple = smf.ols(formula=formula_str, data=df).fit()
print(model_simple.summary())

In [ ]:
# split dataset
train_dataset, test_dataset = train_test_split(df, test_size=0.2, random_state=13)
print("train size: {}, test size: {}".format(len(train_dataset), len(test_dataset)))

In [ ]:
# mixed selection with p-value threshold 0.025
p_thresh = 0.025
# collect feature names
feature_names = []
for col_name in train_dataset.columns:
    if col_name != 'PM':
        feature_names.append(col_name)

formula_str = 'PM ~ '
selected_features = []

num_feat = len(feature_names)
for i in range(num_feat):
    # in each iteration, save the correspinding score of each feature
    r_squared_adj_list = []
    for feat in feature_names:
        formula_str_tmp = concat_formula(formula_str, feat, i)
        model_tmp = smf.ols(formula=formula_str_tmp, data=train_dataset).fit()
        r_squared_adj_list.append((feat, model_tmp.rsquared_adj))
    # sort the features by adjusted R-squared
    r_squared_adj_list = sorted(r_squared_adj_list, key=lambda x: x[1], reverse=True)
    print(r_squared_adj_list)
    # the best feature is the one gives the best adjusted R-squared
    # select the best feature and add it into the model
    best_feat = r_squared_adj_list[0][0]
    feature_names.remove(best_feat)
    selected_features.append(best_feat)
    formula_str = concat_formula(formula_str, best_feat, i)
    
    # check if there are insignificant features in the new model (p-value > threshold)
    while True:
        print("new formula: {}".format(formula_str))
        model_best = smf.ols(formula=formula_str, data=train_dataset).fit()
        # print(model_best.summary())
        pvals = []
        for feat in selected_features:
            pvals.append((feat, model_best.pvalues[feat]))
        pvals = sorted(pvals, key=lambda x: x[1], reverse=True)
        if pvals[0][1] > p_thresh:
            # if the feature with the worst p-value is too bad, it is removed
            selected_features.remove(pvals[0][0])
            formula_str = concat_formula_from_empty(selected_features)
            continue
        else:
            # if the p values of all the features are small enough
            break

# the model obtained from mixed selected
model_selected = smf.ols(formula=formula_str, data=train_dataset).fit()
print(model_selected.summary())

# for the selected features, draw a plot of adjusted R squared scores vs. features added and mse vs. features added
formula_str = 'PM ~ '
r_squared_list_train = []
mse_list_train = []
r_squared_list_test = []
mse_list_test = []
for i in range(len(selected_features)):
    formula_str = concat_formula(formula_str, selected_features[i], i)
    model_tmp_train = smf.ols(formula=formula_str, data=train_dataset).fit()
    r_squared_list_train.append(model_tmp_train.rsquared_adj)
    _, _, _, mse_train = calculate_rsquared_and_mse(model_tmp_train, train_dataset)
    mse_list_train.append(mse_train)
    _, rsquared_adj_test, _, mse_test = calculate_rsquared_and_mse(model_tmp_train, test_dataset)
    r_squared_list_test.append(rsquared_adj_test)
    mse_list_test.append(mse_test)

fig, (ax0, ax1) = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

ax0.set_title("Adjusted R-squared vs. Added Features")
ax0.plot(selected_features, r_squared_list_train, label="train")
ax0.plot(selected_features, r_squared_list_test, label='test')
for tick in ax0.get_xticklabels():
    tick.set_rotation(75)

ax1.set_title("MSE vs. Added Features")
ax1.plot(selected_features, mse_list_train, label='train')
ax1.plot(selected_features, mse_list_test, label='test')
for tick in ax1.get_xticklabels():
    tick.set_rotation(75)

plt.legend()
plt.show()

In [ ]:
# features dropped after mixed selection
features_dropped = ['day', 'year', 'month', 'sin_cbwd', 'TEMP']
# features left after mixed selection
features = ['HUMI', 'cos_cbwd', 'DEWP', 'Iws', 'hour', 'PRES', 'Iprec', 'is_cv']

formula_str = concat_formula_from_empty(features)
model = smf.ols(formula=formula_str, data=train_dataset).fit()
print(model.summary())

In [ ]:
# use VIF threshold = 10, vif scores greater than this threshold means that high correlation exists
vif_thresh = 10

while True:
    vif_dataset = train_dataset[features]
    vif_scores = [VIF(vif_dataset, i) for i in range(len(features))]
    print(vif_scores)
    idx_max = np.argmax(vif_scores)
    if vif_scores[idx_max] > vif_thresh:
        # the worst feature with vif > threshold should be removed 
        print("{} vif: {}, removed".format(features[idx_max], vif_scores[idx_max]))
        features_dropped.append(features[idx_max])
        features.remove(features[idx_max])
    else:
        # all the remaining features are good
        break

print("features: {}".format(features))
formula_str = concat_formula_from_empty(features)
model = smf.ols(formula=formula_str, data=train_dataset).fit()
print(model.summary())

# save the best adjusted R-squared score for later use
rsquared_adj_best = model.rsquared_adj

In [ ]:
# up to order 10
n = 10
orders = list(range(1, n + 1))

for feat in features:
    # for each feature, draw Adjusted R-suqared vs. orders and MSE vs. orders curves
    # then select the best order according to the plot manually
    formula_str = concat_formula_from_empty(features)
    r_squared_list_train = []
    mse_list_train = []
    r_squared_list_test = []
    mse_list_test = []
    
    for k in orders:
        if k == 1:
            pass
        else:
            formula_str = formula_str + ' + np.power({}, {})'.format(feat, k)
        # print(formula_str)
        model_tmp_train = smf.ols(formula=formula_str, data=train_dataset).fit()
        r_squared_list_train.append(model_tmp_train.rsquared_adj)
        _, _, _, mse_train = calculate_rsquared_and_mse(model_tmp_train, train_dataset)
        mse_list_train.append(mse_train)
        _, r_squared_adj_test, _, mse_test = calculate_rsquared_and_mse(model_tmp_train, test_dataset)
        r_squared_list_test.append(r_squared_adj_test)
        mse_list_test.append(mse_test)

    # draw Adjusted R-suqared vs. orders and MSE vs. orders curves
    fig, (ax0, ax1) = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))

    ax0.set_title("Adjusted R-squared vs. Added Features of High Orders ({})".format(feat))
    ax0.plot(orders, r_squared_list_train, label="train")
    ax0.plot(orders, r_squared_list_test, label='test')
    for tick in ax0.get_xticklabels():
        tick.set_rotation(75)
    
    ax1.set_title("MSE vs. Added Features of High Orders ({})".format(feat))
    ax1.plot(orders, mse_list_train, label='train')
    ax1.plot(orders, mse_list_test, label='test')
    for tick in ax1.get_xticklabels():
        tick.set_rotation(75)
    
    plt.legend()
    plt.show()

In [ ]:
# final_features
features_all = features[:]
# high-order terms selected according to the curves output from the previous cell
features_all.append("np.power({}, {})".format('DEWP', 2))
features_all.append("np.power({}, {})".format('DEWP', 3))
features_all.append("np.power({}, {})".format('DEWP', 4))
features_all.append("np.power({}, {})".format('PRES', 2))
features_all.append("np.power({}, {})".format('hour', 2))
features_all.append("np.power({}, {})".format('hour', 3))
features_all.append("np.power({}, {})".format('hour', 4))

# the model with high-order terms added
formula_str = concat_formula_from_empty(features_all)

model = smf.ols(formula=formula_str, data=train_dataset).fit()
print(model.summary())

In [ ]:
# interaction features
features_interactions = []
for i in range(len(features)):
    for j in range(i + 1, len(features)):
        features_interactions.append("{}:{}".format(features[i], features[j]))

num_feat_inter = len(features_interactions)
formula_str = concat_formula_from_empty(features_all)
# using mixed selection to select interaction terms
for i in range(num_feat_inter):
    rsquared_adj_list = []
    for feat_inter in features_interactions:
        formula_str_tmp = concat_formula_from_another(formula_str, [feat_inter])
        # print(formula_str_tmp)
        model_tmp = smf.ols(formula=formula_str_tmp, data=train_dataset).fit()
        rsquared_adj_list.append((feat_inter, model_tmp.rsquared_adj))
    
    # select the interaction term with best adjusted R-squared value from remaining interaction terms
    rsquared_adj_list = sorted(rsquared_adj_list, key=lambda x: x[1], reverse=True)
    best_feat_inter = rsquared_adj_list[0][0]

    # check if insignificant features exist after a new interaction term is added
    formula_str = concat_formula_from_another(formula_str, [best_feat_inter])
    features_all.append(best_feat_inter)
    while True:
        model_tmp = smf.ols(formula=formula_str, data=train_dataset).fit()
    
        pvals = []
        for feat in features_all:
            pvals.append((feat, model_tmp.pvalues[feat]))
        pvals = sorted(pvals, key=lambda x: x[1], reverse=True)

        # find the interaction term with the worst p-value
        worst_feat_inter = None
        worst_feat_inter_pval = 1
        for k in range(len(pvals)):
            if pvals[k][0] in features_interactions:
                worst_feat_inter = pvals[k][0]
                worst_feat_inter_pval = pvals[k][1]
                break
        if worst_feat_inter_pval > p_thresh:
            features_all.remove(worst_feat_inter)
            formula_str = concat_formula_from_empty(features_all)
        else:
            # all p-values of interaction terms are small enough
            break


formula_str = concat_formula_from_empty(features_all)
model = smf.ols(formula=formula_str, data=train_dataset).fit()
print(model.summary())

# check the adjusted R-squared curves and mse curves
r_squared_list_train = []
mse_list_train = []
r_squared_list_test = []
mse_list_test = []

formula_str = 'PM ~ '
for i in range(len(features_all)):
    feat = features_all[i]
    formula_str = concat_formula(formula_str, feat, i)
    model_tmp = smf.ols(formula=formula_str, data=train_dataset).fit()
    r_squared_list_train.append(model_tmp.rsquared_adj)
    _, _, _, mse_train = calculate_rsquared_and_mse(model_tmp, train_dataset)
    mse_list_train.append(mse_train)
    _, rsquared_adj_test, _, mse_test = calculate_rsquared_and_mse(model_tmp, test_dataset)
    r_squared_list_test.append(rsquared_adj_test)
    mse_list_test.append(mse_test)

# for the selected features, draw a plot of adjusted R squared scores vs. features added and MSE vs. features added
fig, (ax0, ax1) = plt.subplots(nrows=1, ncols=2, figsize=(16, 4))

ax0.set_title("Adjusted R-squared vs. Added Features")
ax0.plot(features_all, r_squared_list_train, label="train")
ax0.plot(features_all, r_squared_list_test, label='test')
for tick in ax0.get_xticklabels():
    tick.set_rotation(75)

ax1.set_title("MSE vs. Added Features")
ax1.plot(features_all, mse_list_train, label='train')
ax1.plot(features_all, mse_list_test, label='test')
for tick in ax1.get_xticklabels():
    tick.set_rotation(75)

plt.legend()
plt.show()

In [ ]:
# select the first two interaction features in features_all according to the curves output from the previous cell
features_all = features_all[:17]
formula_str = concat_formula_from_empty(features_all)
model = smf.ols(formula_str, data=train_dataset).fit()
print(model.summary())

reg_rsquared_test, reg_rsquared_adj_test, reg_mse_resid_test, reg_mse_test = calculate_rsquared_and_mse(model, test_dataset)
_, _, _, reg_mse_train = calculate_rsquared_and_mse(model, train_dataset)

reg_rsquared_train = model.rsquared
reg_rsquared_adj_train = model.rsquared_adj
reg_mse_resid_train = model.mse_resid

print("training R-squared: {}".format(reg_rsquared_train))
print("testing R-squared: {}".format(reg_rsquared_test))
print("training adjusted R-squared: {}".format(reg_rsquared_adj_train))
print("testing adjusted R-squared: {}".format(reg_rsquared_adj_test))
print("training mse of residuals: {}".format(reg_mse_resid_train))
print("testing mse of residuals: {}".format(reg_mse_resid_test))
print("training mse: {}".format(reg_mse_train))
print("testing mse: {}".format(reg_mse_test))

In [ ]:
# features selected in 2.2
features8 = ['HUMI', 'cos_cbwd', 'DEWP', 'Iws', 'hour', 'PRES', 'Iprec', 'is_cv']

train_X = train_dataset[features8].values
train_y = train_dataset['PM'].values
test_X = test_dataset[features8].values
test_y = test_dataset['PM'].values
print("train_X shape: {}".format(train_X.shape))
print("train_y shape: {}".format(train_y.shape))

In [ ]:
# large learning rate will break the boosting iterations, use a small learning rate here
lr_hint = 0.01

# select the best loss
losses = ['linear', 'square', 'exponential']
rsquared_list_train = []
mse_list_train = []
rsquared_list_test = []
mse_list_test = []
for l in losses:
    regressor = AdaBoostRegressor(random_state=13, learning_rate=lr_hint, loss=l)
    regressor.fit(train_X, train_y)
    rsquared_train, mse_train = calculate_rsquared_and_mse_adaboost(regressor, train_X, train_y)
    rsquared_test, mse_test = calculate_rsquared_and_mse_adaboost(regressor, test_X, test_y)
    rsquared_list_train.append(rsquared_train)
    mse_list_train.append(mse_train)
    rsquared_list_test.append(rsquared_test)
    mse_list_test.append(mse_test)
    
# draw a plot of R squared scores vs. losses and MSE vs. losses
fig, (ax0, ax1) = plt.subplots(nrows=1, ncols=2, figsize=(16, 4))

ax0.set_title("R-squared vs. Losses")
ax0.plot(losses, rsquared_list_train, label="train")
ax0.plot(losses, rsquared_list_test, label='test')
for tick in ax0.get_xticklabels():
    tick.set_rotation(75)

ax1.set_title("MSE vs. Losses")
ax1.plot(losses, mse_list_train, label='train')
ax1.plot(losses, mse_list_test, label='test')
for tick in ax1.get_xticklabels():
    tick.set_rotation(75)

plt.legend()
plt.show()

In [ ]:
best_loss = 'linear'

# select the best max depth
max_depths = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
rsquared_list_train = []
mse_list_train = []
rsquared_list_test = []
mse_list_test = []
for d in max_depths:
    regressor = AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=d), random_state=13, loss=best_loss, learning_rate=lr_hint)
    regressor.fit(train_X, train_y)
    rsquared_train, mse_train = calculate_rsquared_and_mse_adaboost(regressor, train_X, train_y)
    rsquared_test, mse_test = calculate_rsquared_and_mse_adaboost(regressor, test_X, test_y)
    rsquared_list_train.append(rsquared_train)
    mse_list_train.append(mse_train)
    rsquared_list_test.append(rsquared_test)
    mse_list_test.append(mse_test)
    
# draw a plot of R squared scores vs. max depth and MSE vs. max depth
fig, (ax0, ax1) = plt.subplots(nrows=1, ncols=2, figsize=(16, 4))

ax0.set_title("R-squared vs. Max Depth")
ax0.plot(max_depths, rsquared_list_train, label="train")
ax0.plot(max_depths, rsquared_list_test, label='test')
for tick in ax0.get_xticklabels():
    tick.set_rotation(75)

ax1.set_title("MSE vs. Max Depth")
ax1.plot(max_depths, mse_list_train, label='train')
ax1.plot(max_depths, mse_list_test, label='test')
for tick in ax1.get_xticklabels():
    tick.set_rotation(75)

plt.legend()
plt.show()

In [ ]:
best_max_depth = 8

# since there is a trade-off between learning rate and number of estimators, select a best combination of them
learning_rates = [0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 1]
estimator_nums = [5, 10, 20, 50, 100, 200, 500]
for lr in learning_rates:
    rsquared_list_train = []
    mse_list_train = []
    rsquared_list_test = []
    mse_list_test = []
    for num_est in estimator_nums:
        regressor = AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=best_max_depth), random_state=13,
                                      loss=best_loss, learning_rate=lr, n_estimators=num_est)
        regressor.fit(train_X, train_y)
        rsquared_train, mse_train = calculate_rsquared_and_mse_adaboost(regressor, train_X, train_y)
        rsquared_test, mse_test = calculate_rsquared_and_mse_adaboost(regressor, test_X, test_y)
        rsquared_list_train.append(rsquared_train)
        mse_list_train.append(mse_train)
        rsquared_list_test.append(rsquared_test)
        mse_list_test.append(mse_test)
    
    # convert estimator_nums to strings to be uniformly shown in the plot
    estimator_nums_label = [ str(num_est) for num_est in estimator_nums ]
    
    # draw a plot of R squared scores vs. number of estimators and MSE vs. number of estimators
    fig, (ax0, ax1) = plt.subplots(nrows=1, ncols=2, figsize=(16, 4))
    
    ax0.set_title("R-squared vs. Number of Estimators (lr={})".format(lr))
    ax0.plot(estimator_nums_label, rsquared_list_train, label="train")
    ax0.plot(estimator_nums_label, rsquared_list_test, label='test')
    for tick in ax0.get_xticklabels():
        tick.set_rotation(75)
    
    ax1.set_title("MSE vs. Number of Estimators (lr={})".format(lr))
    ax1.plot(estimator_nums_label, mse_list_train, label='train')
    ax1.plot(estimator_nums_label, mse_list_test, label='test')
    for tick in ax1.get_xticklabels():
        tick.set_rotation(75)
    
    plt.legend()
    plt.show()

In [ ]:
best_lr = 0.2
best_num_estimators = 20

# the final AdaBoostRegressor
ada_regressor = AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=best_max_depth), random_state=13, 
                                      loss=best_loss, learning_rate=best_lr, n_estimators=best_num_estimators)
ada_regressor.fit(train_X, train_y)

# check importance of features in AdaBoostRegressor
result = permutation_importance(ada_regressor, train_X, train_y, n_repeats=10, random_state=0)
print("feature importances:")
print(result['importances_mean'])

# metrics: R-squared and MSE
ada_rsquared_train, ada_mse_train = calculate_rsquared_and_mse_adaboost(ada_regressor, train_X, train_y)
ada_rsquared_test, ada_mse_test = calculate_rsquared_and_mse_adaboost(ada_regressor, test_X, test_y)

print("training R-squared: {}".format(ada_rsquared_train))
print("testing R-squared: {}".format(ada_rsquared_test))
print("training MSE: {}".format(ada_mse_train))
print("testing MSE: {}".format(ada_mse_test))

# plot
methods = ['OLS', 'AdaBoostRegressor']
fig, (ax0, ax1) = plt.subplots(figsize=(10, 10), nrows=2, ncols=2)

ax0[0].set_title("R-squared (training)")
ax0[0].bar(methods, [reg_rsquared_train, ada_rsquared_train])

ax0[1].set_title("R-squared (testing)")
ax0[1].bar(methods, [reg_rsquared_test, ada_rsquared_test])

ax1[0].set_title("MSE (training)")
ax1[0].bar(methods, [reg_mse_train, ada_mse_train])

ax1[1].set_title("MSE (testing)")
ax1[1].bar(methods, [reg_mse_test, ada_mse_test])

plt.show()